In [1]:
import pandas as pd
import seaborn as sns
from pylab import rcParams
import numpy as np
%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

In [2]:
df = pd.read_csv("data/24h_candle.csv", delimiter=',', index_col=0)
df.head()

,id,date_time,open,close,low,high,volume,pair_id
#,,,,,,,,
1,10362847,2015-08-01 07:00:00.000000,284.000000,279.845023,275.724508,284.000000,301.376392,83
2,10362848,2015-08-02 07:00:00.000000,280.000000,286.438075,280.000000,286.438075,106.563859,83
3,10362849,2015-08-03 07:00:00.000000,285.438300,288.965644,284.286275,288.965644,75.706794,83
4,10362850,2015-08-04 07:00:00.000000,281.104486,288.000000,281.104486,288.000000,11.947218,83
5,10362851,2015-08-05 07:00:00.000000,288.200000,287.569800,279.793314,288.965425,9.297527,83


In [3]:
splitFrame = pd.DataFrame(df.date_time.str.split(' ',1).tolist(), columns = ['date','time'],index = df.index)
data = pd.concat([splitFrame, df[df.columns[2:]]], axis=1)

In [4]:
import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

openValue = go.Scatter( x = data.date, y = data.open, name = 'openValue')
closeValues = go.Scatter( x = data.date, y = data.close, name = 'closeValues')
highValue = go.Scatter( x = data.date, y = data.high, name = 'highValue')

iplot([openValue, highValue])

In [67]:
import plotly.graph_objs as go

trace = go.Candlestick(x=data.date,
                       open=data.open,
                       high=data.high,
                       low=data.low,
                       close=data.close,
                      text = data.columns)
layout = {
    'title': 'Candlestick BTC data',
    'yaxis': {'title': 'BTC-USD'},    
}
candleData = [trace]

fig = dict(data=candleData, layout=layout)

iplot(fig)

In [66]:
pd.DataFrame(np.array([[1],[2]]).T, columns = ['date','time'])

,date,time
0,1,2
